# From PostgreSQL to Pandas

*By Naysan Saran, May 2020.*

## 1 - Introduction

In this tutorial we will go through all the steps required to get a Pandas dataframe from an SQL request using Psycopg2. Here we suppose that the SQL table **MonthlyTemp** is already populated. The first rows of the table look like this

In [4]:
import psycopg2
import pandas as pd

Fist, let's specify the connection parameters as a Python dictionary. 

In [5]:
param_dic = {
    "host"      : "localhost",
    "database"  : "globaldata",
    "user"      : "myuser",
    "password"  : "Passw0rd"
}

This function will allow us to connect to the database

In [6]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


Connect to the database

In [7]:
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [12]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df


### Example 1: keeping the original column names

In [13]:
column_names = ["id", "source", "datetime", "mean_temp"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select * from MonthlyTemp", column_names)
df.head()

,id,source,datetime,mean_temp
0,860897,GCAG,2016-12-06,0.7895
1,860898,GISTEMP,2016-12-06,0.8100
2,860899,GCAG,2016-11-06,0.7504
3,860900,GISTEMP,2016-11-06,0.9300
4,860901,GCAG,2016-10-06,0.7292


### Example 2: changing the original column names

In [ ]:
column_names = ["datimestamp", "temperature"]
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select * from MonthlyTemp", column_names)
df.head()

In [12]:
# Close the connection
conn.close()